# 최적화(Optimizing)

### 매개변수에 대한 오류의 도함수를 수집한 뒤 경사하강법을 사용하여 이 파라미터들을 최적화한다.

In [1]:
# 기본 코드
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



# 하이퍼 파라미터란?
-   모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다.
-   서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미친다.

### 학습 시 다음과 같은 하이퍼 파라미터를 정의
-   에폭(epoch) 수 - 데이터셋을 반복하는 횟수
-   배치 크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
-   학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

In [2]:
learning_rate = 1e-3    # 학습률
batch_size = 64         # 배치 크기
epochs = 5              # 에폭 수

# 최적화 단계(Optimization Loop)

-   학습 단계(train loop) - 설정한 에폭만큼 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴합니다.

-   검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)합니다.

# 손실 함수(loss function)

-   손실 함수는 획득한 결과를 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이 값을 최소화하려고 한다.
-   주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산한다.
-   일반적인 손실함수
    - 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))
    - 분류(Classification)에 사용하는 nn.NLLLoss(음의 로그 우도(Negative Log Likelihood))
    - nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss

In [3]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저(Optimizer)

-   각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
-   학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화한다.

### 학습 단계(loop)에서 최적화는 세 단계로 이뤄진다.
-   optimizer.zero_grad() 를 호출하여 모델 매개변수의 기울기를 재설정. 반복할 때마다 명시적으로 0으로 설정
-   loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파
-   기울기를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

In [4]:
# 전체 코드
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301246  [    0/60000]
loss: 2.295998  [ 6400/60000]
loss: 2.276614  [12800/60000]
loss: 2.277495  [19200/60000]
loss: 2.247110  [25600/60000]
loss: 2.220206  [32000/60000]
loss: 2.243629  [38400/60000]
loss: 2.201876  [44800/60000]
loss: 2.199673  [51200/60000]
loss: 2.171570  [57600/60000]
Test Error: 
 Accuracy: 23.9%, Avg loss: 2.165757 

Epoch 2
-------------------------------
loss: 2.173832  [    0/60000]
loss: 2.167004  [ 6400/60000]
loss: 2.112972  [12800/60000]
loss: 2.134125  [19200/60000]
loss: 2.067040  [25600/60000]
loss: 2.018750  [32000/60000]
loss: 2.061383  [38400/60000]
loss: 1.976409  [44800/60000]
loss: 1.986856  [51200/60000]
loss: 1.914358  [57600/60000]
Test Error: 
 Accuracy: 51.1%, Avg loss: 1.910256 

Epoch 3
-------------------------------
loss: 1.941555  [    0/60000]
loss: 1.913023  [ 6400/60000]
loss: 1.800709  [12800/60000]
loss: 1.843728  [19200/60000]
loss: 1.714069  [25600/60000]
loss: 1.675406  [32000/600